In [1]:
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document


# Charger les guides depuis un fichier JSON
def load_guides(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        guides = json.load(f)
    return guides


# Charger les posts depuis un fichier JSON
def load_posts(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        posts = json.load(f)
    return posts


# Convertir les posts et guides en vecteurs et créer un retriever LangChain
def index_data_embeddings(
    posts, guides, model_name="sentence-transformers/all-mpnet-base-v2"
):
    # Construire les textes et les objets Document pour les posts
    documents = []
    for p in posts:
        text_comments = ""
        for comment in p["comments"]:
            text_comments += comment + "\n"
        documents.append(
            Document(
                page_content=f"{p['titre']} - {p['contenu']}",
                metadata={
                    "comments": text_comments,
                    "url": p["url"],
                    "titre": p["titre"],
                    "contenu": p["contenu"],
                },
            )
        )

    # Construire les textes et les objets Document pour les guides
    for g in guides:
        documents.append(
            Document(
                page_content=f"{g['dataType']} - {g['type']} {g['subject']} : {g['title']} {(g['url'])}",
                metadata={
                    "dataType": g["dataType"],
                    "type": g["type"],
                    "subject": g["subject"],
                    "title": g["title"],
                    "category": g["category"],
                    "summary": g["summary"],
                    "url": g["url"],
                    "guideid": g["guideid"],
                },
            )
        )

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(documents)

    # Créer des embeddings avec LangChain
    embedding_model = HuggingFaceEmbeddings(model_name=model_name)
    vector_store = FAISS.from_documents(splits, embedding_model)

    return vector_store.as_retriever(search_kwargs={"k": 2})

In [2]:
posts = load_posts("./data/techsupport_posts.json")

guides = load_guides("./data/guides.json")

retriever = index_data_embeddings(posts, guides)

c:\Users\Coco\Desktop\TB\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from dotenv import load_dotenv
import os

# Charger les variables d'environnement
load_dotenv()
OPENAI_KEY = os.getenv("OPENAI_KEY")

# Génération avec LLM

## Classe pour interagir avec un LLM via Ollama

In [21]:
import ollama
from langchain_core.language_models import LLM
from typing import List


class OllamaLLM(LLM):
    model: str = "mistral"

    def _call(self, prompt: str, stop: List[str] = None) -> str:
        response = ollama.chat(
            model=self.model, messages=[{"role": "user", "content": prompt}]
        )
        return response["message"]["content"]

    @property
    def _identifying_params(self) -> dict:
        return {"model": self.model}

    @property
    def _llm_type(self) -> str:
        return "ollama"

In [5]:
from langchain.prompts import PromptTemplate

# Le prompt pour le modèle
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
L'utilisateur pose la question suivante :

➡️ {question}

Tu disposes ci-dessous de **guides techniques** et de **posts Reddit** pertinents. Ces contenus incluent des descriptions générales, des conseils pratiques, des solutions proposées par la communauté, et parfois des instructions techniques détaillées.

🎯 Ta mission :
- Analyse et synthétise les informations issues des guides techniques **et** des posts Reddit pour répondre à la question.
- Fournis une réponse structurée et complète.
- Utilise les étapes décrites dans les guides techniques, si présentes, et les solutions suggérées par les utilisateurs dans les posts Reddit.
- Répond en **Français**

📚 **Sources disponibles :**

{context}

🛠 **Format de réponse attendu :**

---
🔍 **Analyse du problème** :
[Présente une synthèse du problème posé, en te basant sur les informations extraites des documents.]

✅ **Vérifications préalables recommandées** :
[Liste les éléments à inspecter ou tester avant de commencer les manipulations.]

📝 **Procédure détaillée proposée** :
[Utilise les étapes comme "Step 1", "Step 2" pour les guides iFixit, ou les conseils donnés dans les posts Reddit.]

💡 **Conseils supplémentaires ou précautions à prendre** :
[Ajoute des conseils supplémentaires tirés des guides ou des commentaires des utilisateurs.]

🔗 **Sources consultées** :
[Indique ici les URL des documents (guides ou posts Reddit) ayant servi à construire ta réponse. Utilise les URLs disponibles dans les métadonnées des documents fournis.]
---

🎯 **Important** : Structure ta réponse de manière fluide, concise, et professionnelle. Mentionne les sources utilisées, telles que l'URL du guide ou du post Reddit.
""",
)

In [6]:
import requests


def get_guide_steps(guideid):
    url = f"https://www.ifixit.com/api/2.0/guides/{guideid}"
    response = requests.get(url)

    if response.status_code != 200:
        return {
            "error": f"Échec de récupération du guide {guideid}, code: {response.status_code}"
        }

    data = response.json()
    steps = []

    cpt_steps = 0

    for step in data.get("steps", []):
        cpt_steps += 1
        step_texts = [
            line["text_rendered"]
            for line in step.get("lines", [])
            if "text_rendered" in line
        ]
        steps.append({"stepno": cpt_steps, "text": step_texts})

    return steps

In [7]:
def format_documents(docs):
    formatted_docs = []

    for doc in docs:
        guide_id = doc.metadata.get("guideid")
        if guide_id:
            guide_steps = get_guide_steps(guide_id)
            print(len(guide_steps))
            guide_infos = ""
            for guide in guide_steps:
                step_text = "\n".join(guide["text"])
                guide_infos += "\n" + f"Step {guide['stepno']}:\n" + step_text
            if guide_infos not in doc.page_content:
                doc.page_content += guide_infos

        metadata_text = "\n".join(
            f"{key}: {value}" for key, value in doc.metadata.items()
        )

        formatted_doc = f"""---\n📄 **Contenu** :\n{doc.page_content}\n\n🔖 **Métadonnées** :\n{metadata_text}\n"""
        formatted_docs.append(formatted_doc)

    print("\n\n".join(formatted_docs))
    return "\n\n".join(formatted_docs)

In [22]:
from langchain_openai import ChatOpenAI

llm = OllamaLLM()

# llm = ChatOpenAI(openai_api_key=OPENAI_KEY, model="gpt-4.1", temperature=0.5)

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


# Chaîne RAG
def create_rag_chain(retriever):
    return (
        {
            "context": retriever | format_documents,
            "question": RunnablePassthrough(),
        }
        | prompt_template
        | llm
        | StrOutputParser()
    )

In [ ]:
rag_chain = create_rag_chain(retriever)
question = "Repair Iphone 12 screen"
response = rag_chain.invoke(question)
print(response)

---
📄 **Contenu** :
]right] bracket] key is t]riggered const]t]ant]ly desp]it]e t]he physical key it]self being fine - First] off I want] t]o t]hank anyone t]hat] reads my po]st] as it] will be more challenging to] read t]han most]].

}}Las]t] wint]er I sp]illed beer on my lap]to]p]]]. I} ai]red it] o]ut]], t]ook it] ap]art]], wipe]d] it]] d]o]wn]]]]]]]]],]]]]] a]]nd ]]it] seeme]]d ]t]o ]]]]]]]]]]]]]s]]]]]]]]]]]]]]]]]s]]t]art] working again]]]. ]]E}]]]]x]cept ]t]hat] for]] ]a] few weeks aft]erwards my righ]t] brac]]ket] key was s]]]t]uc]k down]]]]]]]-]-]]n]ot] p]h]y]si]cally,]] m]]ind ]yo]u]]]]]]]]]]]]], but] th]at] for somereason it] j]us]t ]kep]t] ]get]t]ing t]rigger]]ed.]]
]

Which is great]]ly annoying for a mul]t]it]u]de of reasons. }}S}u]dd]]e]]]nly ]]]]t]yp]]]]]]]]]]]]]]l]]]o]]g]]]]g]]i]]n]]]g]] ]i]n]]t]o ]]]a]]cco]un]t]s was a minigame of sp]eed and]] a]g]il]i]t]y.]]]]]]]]]]]]]]]]]]]

🔖 **Métadonnées** :
comments: Pop the key off and clean the contact with a q tip and rubbing a